In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import detrend, butter, filtfilt
import scipy.io
from pathlib import Path

In [2]:
SAMPLE_RATE = 9000
FILE_DIR_INPUT = "raw_data/ball"
FILE_DIR_OUTPUT = "mat_data/filtered/ball"
EXAMPLE_PATH_MAT = "example.mat"

In [3]:
files = []
for file in Path(FILE_DIR_INPUT).iterdir():
    if file.is_file():
        files.append(file)

In [4]:
def read_data_raw(FILE_PATH_RAW):
    header = []
    data = []
    with open(FILE_PATH_RAW, 'r') as file:
        for line in file:
            line = line.strip()
            if line == "Data as Time Sequence:":
                break
            if line:  
                header.append(line)
        
        for line in file:
            line = line.strip()
            if line:  
                try:
                    value = float(line)
                    data.append(value)
                except ValueError:
                    continue  
    return data, header

In [5]:
def find_peak_frequency_in_range(signal, sampling_rate, f_min, f_max):
    n = len(signal)
    freq = np.fft.rfftfreq(n, d=1/sampling_rate)
    fft_values = np.fft.rfft(signal)
    magnitude = np.abs(fft_values)
    mask = (freq >= f_min) & (freq <= f_max)
    freq_in_range = freq[mask]
    magnitude_in_range = magnitude[mask]
    if len(magnitude_in_range) == 0:
        return None, None
    max_index = np.argmax(magnitude_in_range)
    peak_frequency = freq_in_range[max_index]
    peak_amplitude = magnitude_in_range[max_index]
    return peak_frequency, peak_amplitude

In [6]:
def butter_bandstop(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandstop')
    return b, a

def butter_bandstop_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandstop(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [7]:
for file in files:
    signal, header = read_data_raw(file)
    peak_freq, peak_vibr = find_peak_frequency_in_range(signal, SAMPLE_RATE, 1, 30)
    lowcut = peak_freq - 2
    highcut = peak_freq + 2
    filtered_signal = butter_bandstop_filter(signal, lowcut, highcut, SAMPLE_RATE, order=2)
    mat_source = scipy.io.loadmat(EXAMPLE_PATH_MAT)
    mat_source["X118_DE_time"] = np.array([[x] for x in filtered_signal])
    file_name = file.name.split(".")[:-1][0]
    scipy.io.savemat(str(FILE_DIR_OUTPUT) + "/" + file_name + ".mat", mat_source)

/home/maybe/Рабочий стол/github/.venv/lib/python3.12/site-packages/scipy/io/matlab/_mio.py:317: MatWriteWarning: Starting field name with a underscore (__header__) is ignored
  MW.put_variables(mdict)
/home/maybe/Рабочий стол/github/.venv/lib/python3.12/site-packages/scipy/io/matlab/_mio.py:317: MatWriteWarning: Starting field name with a underscore (__version__) is ignored
  MW.put_variables(mdict)
/home/maybe/Рабочий стол/github/.venv/lib/python3.12/site-packages/scipy/io/matlab/_mio.py:317: MatWriteWarning: Starting field name with a underscore (__globals__) is ignored
  MW.put_variables(mdict)
